# Implementation of HOG extractor

- In this notebook, you will work on implementing HOG extractor. There are several **blanks** you need to fill in.
- Extraction of HOG feature from an image proceeds as follows:
    1. computing the gradient image in x and y
    2. computing the magnitude and orientation from the gradients
    3. computing gradient histograms
    4. normalizing across blocks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, exposure
from skimage.color import rgb2gray
from scipy import pi
from scipy.ndimage import uniform_filter

## 1. Computing the gradient image in x and y

In [ ]:
def compute_gradients(image):
    """
    Compute the gradient of the image vertically and horizontally.
    
    Parameters
    ----------
    image : (X, Y) Input image array. 
    
    Returns
    -------
    I_x, I_y: the gradient on x-axis and y-axis.
    """
    
    image = np.atleast_2d(image)
    
    if image.ndim > 2:
        raise ValueError("Currently only supports grey-level images")\
    
    if image.dtype.kind == 'u':
        # convert uint image to float
        # to avoid problems with subtracting unsigned numbers in np.diff()
        image = image.astype('float')
        
    # initialize the parameters.
    I_x = np.zeros(image.shape)
    I_y = np.zeros(image.shape)
    
    ########## START CODE HERE ##########
    diff_x = np.diff(image, n=1, axis=1)
    diff_y = np.diff(image, n=1, axis=0)
    I_x[:, 1:-1] = (diff_x[:, :-1] + diff_x[:, 1:]) / 2 # TODO
    I_y[1:-1, :] = (diff_y[:-1] + diff_y[1:]) / 2 # TODO
    ########## END CODE HERE ##########
    
    return I_x, I_y

In [ ]:
def yan_compute_gradients(image):
    """
    Compute the gradient of the image vertically and horizontally.
    
    Parameters
    ----------
    image : (X, Y) Input image array. 
    
    Returns
    -------
    I_x, I_y: the gradient on x-axis and y-axis.
    """
    
    image = np.atleast_2d(image)
    
    if image.ndim > 2:
        raise ValueError("Currently only supports grey-level images")\
    
    if image.dtype.kind == 'u':
        # convert uint image to float
        # to avoid problems with subtracting unsigned numbers in np.diff()
        image = image.astype('float')
        
    # initialize the parameters.
    diff_x = np.zeros(image.shape)
    diff_y = np.zeros(image.shape)
    
    ########## START CODE HERE ##########
    diff_x[:, :-1] = np.diff(image, n=1, axis=1)
    diff_y[:-1, :] = np.diff(image, n=1, axis=0)
    ########## END CODE HERE ##########
    
    return diff_x, diff_y

In [ ]:
# test 1
image = np.arange(16).reshape(4, 4)
I_x, I_y = compute_gradients(image)


assert (I_x == np.array([[0, 1, 1, 0]]* 4)).all()
assert (I_y == np.array([[0, 0, 0, 0], [4, 4, 4, 4], [4, 4, 4, 4], [0, 0, 0, 0]])).all()

## 2. computing the magnitude and orientation from the gradients

- Now we have the gradients $I_x, I_y$. Let's calculate magnitude $m$ and orientation $\theta$ from the gradients.
- Note that magnitude $m$ and orientation $\theta$ are calculated as follows:
$$
m(x,y) = \sqrt{{I_x(x,y)}^2 + {I_y(x,y)}^2}\\
\theta(x,y) = \tan^{-1}\frac{I_y(x,y)}{I_x(x,y)} \times \frac{180^\circ}{\pi}\ \ \  (0^\circ \le \theta < 180^\circ)
$$
- $\theta$ is expressed in **degrees**

In [ ]:
def compute_magnitude_and_orientation(I_x, I_y):
    """
    Compute the magnitude and orientation from the gradients I_x, I_y.
    
    Parameters
    ----------
    I_x: the gradient on x-axis.
    I_y: the gradient on y-axis.
    
    Returns
    -------
    m: magnitude of the gradient.
    orientation: orientation of the gradient.
    """
    
    magnitude = np.sqrt(I_x**2 + I_y**2) # TODO
    orientation = np.arctan2(I_y, I_x) * (180 / pi) % 180 # TODO

    return magnitude, orientation

In [ ]:
# test 1
# image = np.arange(16).reshape(4, 4)
I_x, I_y = compute_gradients(image)
magnitude, orientation = compute_magnitude_and_orientation(I_x, I_y)
expected_magnitude = np.load('./arrays_for_testing/magnitude_1.npy')
expected_orientation = np.load('./arrays_for_testing/orientation_1.npy')
assert np.sum((magnitude - expected_magnitude)**2) < 1e-3
assert np.sum((orientation - expected_orientation)**2) < 1e-3

# test 2
## thetaが180を超えるやつ

## 3.Computing gradient histograms

6行目： N_p//2 から N_p * n_cells_y まで N_p ごとにサンプリングしていく。
line 6: sample from N_p//2 to N_p * n_cells_y at every N_p.

9~13行目： 量子化したいthetaの角度内ならそのまま、その他の箇所を -1 にする。
line 9~13: if theta is not in range of what we want to quantize make it to -1.

16~17行目: 上と同じように量子化したいtheta内にない場合、そこのmagnitudeを0にする。
line 16~17: make magnitude to zero of what it's not in range of what we want to quantize.

20行目: N_p x N_p で平均をとる。
line 20: avarage cells in N_p x N_p.

23行目: 6行目の通りにサンプリングを行う。（サンプリングされた値は20行目よりN_p x N_p内のセルの平均値）
line 23: sample using line 6. (because of line 20 this will be sampling avarage of N_p x N_p)

In [ ]:
def compute_gradient_histograms(magnitude, orientation, N_theta, N_p):

    orientation_histogram = np.zeros((n_cells_x, n_cells_y, N_theta))
    
#     generate (slice(start_pixel_y, end_pixel_y, N_p), slice(start_pixel_x, end_pixel_x, N_p))
    subsample = np.index_exp[N_p//2:N_p * n_cells_y: N_p,
                            N_p//2:N_p * n_cells_x: N_p,]

    for i in range(N_theta):
        temp_ori = np.where(orientation < 180.0 / N_theta * (i + 1),
                           orientation, -1)
        temp_ori = np.where(orientation >= 180.0 / N_theta * i,
                            temp_ori, -1)

#         select magnitudes for those N_theta
        current_theta = temp_ori > -1
        temp_mag = np.where(current_theta, magnitude, 0)

#         normalize in N_p * N_p
        temp_filt = uniform_filter(temp_mag, size=(N_p, N_p))

#         sample cell in the middle
        orientation_histogram[:, :, i] = temp_filt[subsample]
    return orientation_histogram 

## 4. Normalizing across blocks   

セル内を近傍のセルを使って正規化。
教科書 式 2.78を実装。

normalize cells using nearby cells.
implementation of formula 2.78 in handbook.

In [ ]:
def normalize_across_blocks(orientation_histogram):
    normalised_blocks = np.zeros((n_blocks_y, n_blocks_x,
                                  N_c, N_c, N_theta))

    for x in range(n_blocks_x):
        for y in range(n_blocks_y):
            block = orientation_histogram[y:y + N_c, x:x + N_c, :]
            eps = 1e-5
    ########## START CODE HERE ##########
            normalised_blocks[y, x, :] = block / np.sqrt(block.sum()**2 + eps)
    ########## END CODE HERE ##########
    return normalised_blocks

### run all

In [ ]:
N_theta = 2
N_p = 3
N_c = 1
image = np.arange(36).reshape(6, 6)

sy, sx = image.shape
n_cells_x = int(np.floor(sx//N_p))
n_cells_y = int(np.floor(sy//N_p))

n_blocks_x = n_cells_x - N_c + 1
n_blocks_y = n_cells_y - N_c + 1

In [ ]:
def main():
    
    gx, gy = yan_compute_gradients(image)
    
    magnitude, orientation = compute_magnitude_and_orientation(gx, gy)
    
    orientation_histogram = compute_gradient_histograms(magnitude, orientation, N_theta, N_p)
    
    normalised_blocks = normalize_across_blocks(orientation_histogram)

main()